# TransUNet Implementation Pipeline

This notebook consolidates the entire pipeline for the TransUNet project, including:
1.  **Configuration**: Setting up paths and parameters.
2.  **Preprocessing**: Splitting and converting the Synapse dataset.
3.  **Model Definition**: Implementing the TransUNet architecture (ResNet-50 + ViT).
4.  **Metrics**: Defining evaluation metrics (Dice, IoU, HD95).
5.  **Dataset & Loss**: Creating the dataset loader and loss function.
6.  **Training**: Training the model.
7.  **Testing**: Evaluating the trained model on the test set.

## 1. Imports and Configuration

In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
from torchvision import transforms
import SimpleITK as sitk
from tqdm import tqdm
from scipy.spatial.distance import directed_hausdorff
import argparse

# Configuration
BASE_DIR = r"C:\Users\yuvar\Projects\Computer Vision\Project\synapse_data\Abdomen\RawData\Training"
OUTPUT_DIR = r"C:\Users\yuvar\Projects\Computer Vision\Project\data\preprocessed"
MODEL_SAVE_DIR = r"C:\Users\yuvar\Projects\Computer Vision\Project\models"
IMG_DIR = os.path.join(BASE_DIR, "img")
LABEL_DIR = os.path.join(BASE_DIR, "label")

TRAIN_SIZE = 18
TEST_SIZE = 12
NUM_CLASSES = 14
IMG_SIZE = 224
BATCH_SIZE = 24
BASE_LR = 0.01
EPOCHS = 300
SEED = 42

if not os.path.exists(MODEL_SAVE_DIR):
    os.makedirs(MODEL_SAVE_DIR)

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


## 2. Data Preprocessing

In [2]:
def make_folder(path):
    if not os.path.exists(path):
        os.makedirs(path)

def sort_by_number(files, prefix):
    return sorted([f for f in files if f.startswith(prefix)], key=lambda x: int(''.join(filter(str.isdigit, x))))

def save_slice(img, label, split, case_name, slice_idx):
    case_id = case_name.replace(".nii.gz", "").replace("img", "")
    filename = f"case{case_id}_slice{slice_idx:03d}.npz"
    save_path = os.path.join(OUTPUT_DIR, split, filename)
    np.savez(save_path, image=img, label=label)

def process_and_save(img_list, label_list, split):
    slice_count = 0
    for img_name, label_name in tqdm(zip(img_list, label_list), total=len(img_list), desc=f"Processing {split}"):
        img_path = os.path.join(IMG_DIR, img_name)
        label_path = os.path.join(LABEL_DIR, label_name)

        itk_img = sitk.ReadImage(img_path)
        itk_label = sitk.ReadImage(label_path)

        img_array = sitk.GetArrayFromImage(itk_img)
        label_array = sitk.GetArrayFromImage(itk_label)

        # Normalize image: clip to [-125, 275], then normalize to [0, 1]
        img_array = np.clip(img_array, -125, 275)
        img_array = (img_array - (-125)) / (275 - (-125))

        for i in range(img_array.shape[0]):
            img_slice = img_array[i, :, :]
            label_slice = label_array[i, :, :]

            if split == "train":
                if np.sum(label_slice) > 0:
                     save_slice(img_slice, label_slice, split, img_name, i)
                     slice_count += 1
            else:
                save_slice(img_slice, label_slice, split, img_name, i)
                slice_count += 1
    
    print(f"Saved {slice_count} slices for {split}.")

def preprocess_dataset():
    if os.path.exists(os.path.join(OUTPUT_DIR, "train")) and len(os.listdir(os.path.join(OUTPUT_DIR, "train"))) > 0:
        print("Data already preprocessed. Skipping.")
        return

    make_folder(os.path.join(OUTPUT_DIR, "train"))
    make_folder(os.path.join(OUTPUT_DIR, "test"))

    img_files = sort_by_number(os.listdir(IMG_DIR), "img")
    label_files = sort_by_number(os.listdir(LABEL_DIR), "label")

    assert len(img_files) == len(label_files), "Mismatch in image and label count"
    
    train_imgs = img_files[:TRAIN_SIZE]
    train_labels = label_files[:TRAIN_SIZE]
    test_imgs = img_files[TRAIN_SIZE:TRAIN_SIZE+TEST_SIZE]
    test_labels = label_files[TRAIN_SIZE:TRAIN_SIZE+TEST_SIZE]

    print(f"Training set: {len(train_imgs)} scans")
    print(f"Testing set: {len(test_imgs)} scans")

    process_and_save(train_imgs, train_labels, "train")
    process_and_save(test_imgs, test_labels, "test")

# Run preprocessing
preprocess_dataset()

Training set: 18 scans
Testing set: 12 scans


Processing train: 100%|██████████| 18/18 [00:08<00:00,  2.25it/s]


Saved 1338 slices for train.


Processing test: 100%|██████████| 12/12 [00:06<00:00,  1.79it/s]

Saved 1454 slices for test.


## 3. Model Definition (TransUNet)

In [3]:
class Conv2dReLU(nn.Sequential):
    def __init__(self, in_channels, out_channels, kernel_size, padding=0, stride=1, use_batchnorm=True):
        conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride=stride, padding=padding, bias=not (use_batchnorm))
        relu = nn.ReLU(inplace=True)
        bn = nn.BatchNorm2d(out_channels) if use_batchnorm else nn.Identity()
        super(Conv2dReLU, self).__init__(conv, bn, relu)

class ResNetEncoder(nn.Module):
    def __init__(self, pretrained=True):
        super().__init__()
        resnet = models.resnet50(pretrained=pretrained)
        self.conv1 = resnet.conv1
        self.bn1 = resnet.bn1
        self.relu = resnet.relu
        self.maxpool = resnet.maxpool
        self.layer1 = resnet.layer1
        self.layer2 = resnet.layer2
        self.layer3 = resnet.layer3
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)      # 1/4
        
        x1 = self.layer1(x)      # 1/4, 256
        x2 = self.layer2(x1)     # 1/8, 512
        x3 = self.layer3(x2)     # 1/16, 1024
        
        return x1, x2, x3

class ViTBlock(nn.Module):
    def __init__(self, hidden_dim, heads, mlp_dim, dropout=0.1):
        super().__init__()
        self.norm1 = nn.LayerNorm(hidden_dim)
        self.attn = nn.MultiheadAttention(hidden_dim, heads, dropout=dropout, batch_first=True)
        self.norm2 = nn.LayerNorm(hidden_dim)
        self.mlp = nn.Sequential(
            nn.Linear(hidden_dim, mlp_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(mlp_dim, hidden_dim),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        attn_out, _ = self.attn(self.norm1(x), self.norm1(x), self.norm1(x))
        x = x + attn_out
        x = x + self.mlp(self.norm2(x))
        return x

class ViT(nn.Module):
    def __init__(self, input_dim=1024, hidden_dim=768, num_layers=12, heads=12, mlp_dim=3072, num_patches=196):
        super().__init__()
        self.embedding = nn.Linear(input_dim, hidden_dim)
        self.pos_embedding = nn.Parameter(torch.zeros(1, num_patches, hidden_dim))
        self.blocks = nn.ModuleList([
            ViTBlock(hidden_dim, heads, mlp_dim) for _ in range(num_layers)
        ])
        self.norm = nn.LayerNorm(hidden_dim)

    def forward(self, x):
        B, C, H, W = x.shape
        x = x.flatten(2).transpose(1, 2) # (B, N, C)
        
        x = self.embedding(x)
        x = x + self.pos_embedding[:, :x.size(1), :]
        
        for block in self.blocks:
            x = block(x)
        
        x = self.norm(x)
        x = x.transpose(1, 2).reshape(B, -1, H, W)
        return x

class DecoderBlock(nn.Module):
    def __init__(self, in_channels, skip_channels, out_channels):
        super().__init__()
        self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.conv = nn.Sequential(
            Conv2dReLU(in_channels + skip_channels, out_channels, kernel_size=3, padding=1),
            Conv2dReLU(out_channels, out_channels, kernel_size=3, padding=1)
        )

    def forward(self, x, skip=None):
        x = self.up(x)
        if skip is not None:
            if x.shape != skip.shape:
                x = nn.functional.interpolate(x, size=skip.shape[2:], mode='bilinear', align_corners=True)
            x = torch.cat([x, skip], dim=1)
        return self.conv(x)

class TransUNet(nn.Module):
    def __init__(self, num_classes=9, img_dim=224):
        super().__init__()
        self.encoder = ResNetEncoder()
        self.vit = ViT(input_dim=1024, hidden_dim=768, num_layers=12, heads=12, mlp_dim=3072, num_patches=196)
        
        self.decoder1 = DecoderBlock(768, 512, 256)
        self.decoder2 = DecoderBlock(256, 256, 128)
        self.decoder3 = DecoderBlock(128, 0, 64)
        self.decoder4 = DecoderBlock(64, 0, 16)
        
        self.final_conv = nn.Conv2d(16, num_classes, kernel_size=1)
        
    def forward(self, x):
        x1, x2, x3 = self.encoder(x)
        x_vit = self.vit(x3)
        
        d1 = self.decoder1(x_vit, x2)
        d2 = self.decoder2(d1, x1)
        d3 = self.decoder3(d2)
        d4 = self.decoder4(d3)
        
        out = self.final_conv(d4)
        return out

## 4. Evaluation Metrics

In [4]:
def dice_coefficient(pred, target, smooth=1e-5):
    if isinstance(pred, torch.Tensor):
        pred = pred.detach().cpu().numpy()
    if isinstance(target, torch.Tensor):
        target = target.detach().cpu().numpy()
        
    pred_flat = pred.flatten()
    target_flat = target.flatten()
    
    intersection = (pred_flat * target_flat).sum()
    return (2. * intersection + smooth) / (pred_flat.sum() + target_flat.sum() + smooth)

def iou_score(pred, target, smooth=1e-5):
    if isinstance(pred, torch.Tensor):
        pred = pred.detach().cpu().numpy()
    if isinstance(target, torch.Tensor):
        target = target.detach().cpu().numpy()
        
    pred_flat = pred.flatten()
    target_flat = target.flatten()
    
    intersection = (pred_flat * target_flat).sum()
    union = pred_flat.sum() + target_flat.sum() - intersection
    return (intersection + smooth) / (union + smooth)

def hausdorff_distance_95(pred, target):
    if isinstance(pred, torch.Tensor):
        pred = pred.detach().cpu().numpy()
    if isinstance(target, torch.Tensor):
        target = target.detach().cpu().numpy()
        
    if np.sum(pred) == 0 or np.sum(target) == 0:
        return 0.0
        
    pred_coords = np.argwhere(pred)
    target_coords = np.argwhere(target)
    
    hd_1 = directed_hausdorff(pred_coords, target_coords)[0]
    hd_2 = directed_hausdorff(target_coords, pred_coords)[0]
    
    return max(hd_1, hd_2)

def compute_metrics(pred, target, num_classes=9):
    if pred.ndim == 4:
        pred = torch.argmax(pred, dim=1)
        
    if isinstance(pred, torch.Tensor):
        pred = pred.detach().cpu().numpy()
    if isinstance(target, torch.Tensor):
        target = target.detach().cpu().numpy()
        
    metrics = {'Dice': [], 'IoU': [], 'HD95': []}
    
    for c in range(1, num_classes):
        pred_c = (pred == c).astype(np.float32)
        target_c = (target == c).astype(np.float32)
        
        metrics['Dice'].append(dice_coefficient(pred_c, target_c))
        metrics['IoU'].append(iou_score(pred_c, target_c))
        
    return {k: np.mean(v) for k, v in metrics.items() if v}

## 5. Dataset and Loss Function

In [5]:
class SynapseDataset(Dataset):
    def __init__(self, base_dir, split="train", transform=None):
        self.base_dir = base_dir
        self.split = split
        self.transform = transform
        self.sample_list = []
        
        split_dir = os.path.join(self.base_dir, split)
        if os.path.exists(split_dir):
            self.sample_list = os.listdir(split_dir)
        
        print(f"Loaded {len(self.sample_list)} samples for {split}")

    def __len__(self):
        return len(self.sample_list)

    def __getitem__(self, idx):
        slice_name = self.sample_list[idx]
        data_path = os.path.join(self.base_dir, self.split, slice_name)
        data = np.load(data_path)
        image, label = data['image'], data['label']
        
        image = torch.from_numpy(image).float()
        label = torch.from_numpy(label).long()
        
        image = image.unsqueeze(0)
        image = image.repeat(3, 1, 1)
        
        target_size = (224, 224)
        if image.shape[-2:] != target_size:
            image = torch.nn.functional.interpolate(image.unsqueeze(0), size=target_size, mode='bilinear', align_corners=False).squeeze(0)
            label = torch.nn.functional.interpolate(label.unsqueeze(0).unsqueeze(0).float(), size=target_size, mode='nearest').long().squeeze(0).squeeze(0)
        
        sample = {'image': image, 'label': label}
        
        if self.transform:
            sample = self.transform(sample)
            
        return sample

class DiceCELoss(nn.Module):
    def __init__(self, num_classes=9):
        super().__init__()
        self.ce = nn.CrossEntropyLoss()
        self.num_classes = num_classes
        
    def forward(self, pred, target):
        ce_loss = self.ce(pred, target)
        
        pred_softmax = torch.softmax(pred, dim=1)
        dice_loss = 0
        for c in range(1, self.num_classes):
            pred_c = pred_softmax[:, c, :, :]
            target_c = (target == c).float()
            dice_loss += (1 - dice_coefficient(pred_c, target_c))
            
        dice_loss /= (self.num_classes - 1)
        
        return 0.5 * ce_loss + 0.5 * dice_loss

## 6. Training Loop

In [6]:
import csv
import os

def train_model():
    # Data
    train_dataset = SynapseDataset(OUTPUT_DIR, split="train")
    # Split train into train/val
    train_size = int(0.8 * len(train_dataset))
    val_size = len(train_dataset) - train_size
    train_subset, val_subset = torch.utils.data.random_split(train_dataset, [train_size, val_size])
    
    train_loader = DataLoader(train_subset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
    val_loader = DataLoader(val_subset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
    
    # Model
    model = TransUNet(num_classes=NUM_CLASSES, img_dim=IMG_SIZE).to(device)
    
    # Optimizer
    optimizer = optim.SGD(model.parameters(), lr=BASE_LR, momentum=0.9, weight_decay=1e-4)
    criterion = DiceCELoss(num_classes=NUM_CLASSES)
    
    best_dice = 0.0
    
    # Setup logging
    log_file = os.path.join(MODEL_SAVE_DIR, "training_log.csv")
    with open(log_file, mode='w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['Epoch', 'Train Loss', 'Val Dice', 'Best Dice'])
    
    print("Starting Training...")
    for epoch in range(EPOCHS):
        model.train()
        train_loss = 0
        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS} [Train]"):
            images = batch['image'].to(device)
            labels = batch['label'].to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            
        # Validation
        model.eval()
        val_dice = 0
        with torch.no_grad():
            for batch in tqdm(val_loader, desc=f"Epoch {epoch+1}/{EPOCHS} [Val]"):
                images = batch['image'].to(device)
                labels = batch['label'].to(device)
                outputs = model(images)
                
                metrics = compute_metrics(outputs, labels, num_classes=NUM_CLASSES)
                val_dice += metrics['Dice']
        
        avg_train_loss = train_loss / len(train_loader)
        avg_val_dice = val_dice / len(val_loader)
        
        print(f"Epoch {epoch+1}: Train Loss: {avg_train_loss:.4f}, Val Dice: {avg_val_dice:.4f}")
        
        if avg_val_dice > best_dice:
            best_dice = avg_val_dice
            torch.save(model.state_dict(), os.path.join(MODEL_SAVE_DIR, "best_model.pth"))
            print("Saved best model.")
            
        # Log to CSV
        with open(log_file, mode='a', newline='') as f:
            writer = csv.writer(f)
            writer.writerow([epoch + 1, avg_train_loss, avg_val_dice, best_dice])

# Uncomment to run training
# train_model()

In [7]:
train_model()

c:\Users\yuvar\anaconda3\envs\cvlab\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\yuvar\anaconda3\envs\cvlab\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loaded 1338 samples for train
Starting Training...


Epoch 1/300 [Val]: 100%|██████████| 12/12 [00:08<00:00,  1.37it/s]


Epoch 1: Train Loss: 1.2095, Val Dice: 0.0810
Saved best model.


Epoch 2/300 [Val]: 100%|██████████| 12/12 [00:05<00:00,  2.39it/s]


...

Epoch 298: Train Loss: 0.0819, Val Dice: 0.8209


Epoch 299/300 [Val]: 100%|██████████| 12/12 [00:04<00:00,  2.45it/s]


Epoch 299: Train Loss: 0.0846, Val Dice: 0.8184


Epoch 300/300 [Val]: 100%|██████████| 12/12 [00:05<00:00,  2.30it/s]

Epoch 300: Train Loss: 0.0841, Val Dice: 0.8208


## 7. Testing and Evaluation

In [8]:
def test_model():
    # Data
    test_dataset = SynapseDataset(OUTPUT_DIR, split="test")
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=0)
    
    # Model
    model = TransUNet(num_classes=NUM_CLASSES, img_dim=IMG_SIZE).to(device)
    model_path = os.path.join(MODEL_SAVE_DIR, "best_model.pth")
    
    if not os.path.exists(model_path):
        print("No model found. Please train first.")
        return
        
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()
    
    organ_metrics = {c: {'Dice': [], 'IoU': [], 'HD95': []} for c in range(1, NUM_CLASSES)}
    
    print("Starting inference...")
    with torch.no_grad():
        for batch in tqdm(test_loader):
            image = batch['image'].to(device)
            label = batch['label'].to(device)
            
            output = model(image)
            pred = torch.argmax(torch.softmax(output, dim=1), dim=1)
            
            for c in range(1, NUM_CLASSES):
                pred_c = (pred == c).cpu().numpy()
                label_c = (label == c).cpu().numpy()
                
                if np.sum(label_c) > 0:
                    dice = dice_coefficient(pred_c, label_c)
                    iou = iou_score(pred_c, label_c)
                    hd95 = hausdorff_distance_95(pred_c, label_c)
                    
                    organ_metrics[c]['Dice'].append(dice)
                    organ_metrics[c]['IoU'].append(iou)
                    organ_metrics[c]['HD95'].append(hd95)
    
    print("\n=== Final Results ===")
    print(f"{'Class':<10} {'Dice':<10} {'IoU':<10} {'HD95':<10}")
    
    avg_dice = []
    avg_iou = []
    avg_hd95 = []
    
    for c in range(1, NUM_CLASSES):
        if organ_metrics[c]['Dice']:
            d = np.mean(organ_metrics[c]['Dice'])
            i = np.mean(organ_metrics[c]['IoU'])
            h = np.mean(organ_metrics[c]['HD95'])
            
            print(f"{c:<10} {d:.4f}     {i:.4f}     {h:.4f}")
            
            avg_dice.append(d)
            avg_iou.append(i)
            avg_hd95.append(h)
        else:
            print(f"{c:<10} N/A        N/A        N/A")
            
    print("-" * 40)
    print(f"{'Average':<10} {np.mean(avg_dice):.4f}     {np.mean(avg_iou):.4f}     {np.mean(avg_hd95):.4f}")

# Uncomment to run testing
# test_model()

In [9]:
test_model()

Loaded 1454 samples for test
Starting inference...


100%|██████████| 1454/1454 [01:22<00:00, 17.62it/s]


=== Final Results ===
Class      Dice       IoU        HD95      
1          0.8443     0.7824     13.5416
2          0.8542     0.7820     8.0282
3          0.8503     0.7769     6.2237
4          0.3314     0.2636     5.6462
5          0.6165     0.4809     4.2824
6          0.8775     0.8247     22.3126
7          0.6093     0.5080     24.4439
8          0.8525     0.7591     4.3959
9          0.7221     0.6020     5.2200
10         0.4564     0.3249     27.9489
11         0.4160     0.3004     19.6295
12         0.2043     0.1352     3.3432
13         0.1528     0.1006     4.2435
----------------------------------------
Average    0.5991     0.5108     11.4815
